In [32]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *

See /nfs/home/genovese/thesis-wildfire-genovese/outputs/EDA_ProfileReport.html for the EDA, here the consequent preprocessing

In [2]:
data = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/database/model_input/dataset_with_days_enumerated.csv')

### 1. drop duplicates and constant variables

In [3]:
data.drop_duplicates(inplace=True)
data.drop(columns=['VEGETAZIONE_TIPO_Saliceto paludoso di Salix cinerea',
                   'VEGETAZIONE_TIPO_Larici-cembreto su rodoreto-vaccinieto',
                   'VEGETAZIONE_INTERVENTO_Gestione a governo misto'],
                   inplace=True)

### 2. Check for almost constant variables

In [4]:
for c in data.columns:
    vals = data[c].value_counts().sort_values(ascending = False)
    if vals.iloc[1:].sum() <= 50:
        print(f'Column {c} has value counts: ', vals)

Column Superficie a funghi has value counts:  Superficie a funghi
0.00    3577
0.10      13
0.32      11
0.01       6
0.75       1
0.05       1
0.03       1
1.00       1
Name: count, dtype: int64
Column VEGETAZIONE_CATEGORIA_Pinete di pino marittimo has value counts:  VEGETAZIONE_CATEGORIA_Pinete di pino marittimo
0.000000    4211
0.026747       1
0.029987       1
0.012061       1
0.007983       1
0.122738       1
0.034664       1
0.055317       1
0.094448       1
0.047102       1
0.053307       1
0.011757       1
0.023077       1
0.017146       1
0.049659       1
0.018880       1
Name: count, dtype: int64
Column VEGETAZIONE_CATEGORIA_Pinete di pino montano has value counts:  VEGETAZIONE_CATEGORIA_Pinete di pino montano
0.000000    4194
0.377300       1
0.012535       1
0.007571       1
0.039844       1
0.002276       1
0.002365       1
0.002382       1
0.194533       1
0.188670       1
0.232105       1
0.120408       1
0.123174       1
0.078491       1
0.068163       1
0.008970       

In [ ]:
# data['TOTALE_VEGETAZIONE'] = data.loc[:, [col for col in data.columns if 'VEGETAZIONE_TIPO' in col]].sum(axis=1)
# data['TOTALE_STRADE'] = data.loc[:, [col for col in data.columns if 'RTP' in col]].sum(axis=1)

In [6]:
to_drop = []

for c in data.columns:
    vals = data[c].value_counts().sort_values(ascending = False)
    if vals.iloc[1:].sum() <= 10:
        if np.max(list(vals.index)) < 0.1:
            to_drop.append(c)

In [ ]:
# data.drop(columns=to_drop, inplace=True)

### 3. Look for too high correlations and drop under necessity

In [ ]:
def find_high_corr_pairs(df, threshold=0.9):
    corr = df.corr()  # compute correlation matrix
    # Extract only upper triangle (exclude duplicates and self-correlation)
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    
    # Stack into long format and filter
    high_corr = upper.stack().loc[lambda x: x.abs() > threshold]
    # Result is a Series with MultiIndex (col1, col2) and value = correlation

    return high_corr

In [120]:
high_corr = find_high_corr_pairs(data, threshold=0.95)
for s in high_corr.index:
    print(s)

('analfabeti', 'alfabeti privi di titolo di studio')
('analfabeti', 'licenza di scuola elementare')
('analfabeti', 'licenza di scuola media inferiore o di avviamento professionale')
('analfabeti', 'diploma di istruzione secondaria di II grado o di qualifica professionale (corso di 3-4 anni) compresi IFTS')
('analfabeti', 'diploma di tecnico superiore ITS o titolo di studio terziario di primo livello')
('analfabeti', 'titolo di studio terziario di secondo livello e dottorato di ricerca')
('analfabeti', 'titolo di studio terziario di secondo livello')
('analfabeti', 'dottorato di ricerca/diploma accademico di formazione alla ricerca')
('analfabeti', 'totale_istruzione')
('analfabeti', '9_24_nessun titolo di studio')
('analfabeti', '9_24_licenza di scuola elementare')
('analfabeti', '9_24_licenza di scuola media inferiore o di avviamento professionale')
('analfabeti', '9_24_diploma di istruzione secondaria di II grado o di qualifica professionale (corso di 3-4 anni) compresi IFTS')
('anal

In [13]:
save_clean_data(data, 'processed_not_imputed_non_rekriged', '/nfs/home/genovese/thesis-wildfire-genovese/database/cache')

File saved to: /nfs/home/genovese/thesis-wildfire-genovese/database/cache/processed_not_imputed_non_rekriged
File saved to: /nfs/home/genovese/thesis-wildfire-genovese/database/cache/processed_not_imputed_rekriged


### 4. Reintroduce target

In [21]:
kriged = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest/sentinel_negative_fires/ambiguous.geojson')
negatives = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest/sentinel_negative_fires/negatives.geojson')

In [10]:
data['target'] = [None]*data.shape[0]

for idx, target in zip(kriged.id, kriged.label):
    j = data[data.fire_id == idx].index
    data.loc[j, 'target'] = target

for idx in negatives.id:
    j = data[data.fire_id == idx].index
    data.loc[j, 'target'] = 0

data.loc[:, 'target'] = data.target.fillna(1)

/tmp/ipykernel_2623385/626330105.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.loc[:, 'target'] = data.target.fillna(1)


In [11]:
import pickle 
with open('/nfs/home/genovese/thesis-wildfire-genovese/src/trained_models/target_kriging.pkl', 'rb') as f:
    krige = pickle.load(f)

In [12]:
from shapely import wkt

data2 = data.copy(deep=True)
for idx, target in zip(negatives.id, krige.predict(np.array(pd.concat([negatives.geometry.centroid.x, negatives.geometry.centroid.y], axis=1)))):
    j = data2[data2.fire_id == idx].index
    data2.loc[j, 'target'] = target

positives = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/database/model_input/table_data_input.csv').drop(index=0)
positives['ignition_point (epsg=3857)'] = positives['ignition_point (epsg=3857)'].apply(wkt.loads)
positives = gpd.GeoDataFrame(positives, geometry='ignition_point (epsg=3857)', crs = 'EPSG:3857')
for idx, target in zip(positives.fire_id, krige.predict(np.array(pd.concat([positives.geometry.x, positives.geometry.y], axis=1)))):
    j = data2[data2.fire_id == idx].index
    data2.loc[j, 'target'] = target

In [ ]:
save_clean_data(data, 'processed_not_imputed_non_rekriged', '/nfs/home/genovese/thesis-wildfire-genovese/database/cache')
save_clean_data(data2, 'processed_not_imputed_rekriged', '/nfs/home/genovese/thesis-wildfire-genovese/database/cache')

### 5. NA imputing
see knn_imputer.py

### 6. Assign target interval

In [33]:
data1 = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/database/model_input/imputed_dataset_non_kriged.csv')
data2 = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/database/model_input/imputed_dataset_kriged.csv')

In [34]:
def classify_target(row, target='target'):
    if row[target] <= 0.2:
        return 0
    elif (row[target] > 0.2) and (row[target] <= 0.4):
        return 1
    elif (row[target] > 0.4) and (row[target] <= 0.6):
        return 2
    elif (row[target] > 0.6) and (row[target] <= 0.8):
        return 3
    elif row[target] > 0.8:
        return 4
    
data1['label'] = data1.apply(lambda x: classify_target(x), axis=1)
data2['label'] = data2.apply(lambda x: classify_target(x), axis=1)

In [30]:
(data1.label == data2.label).all()

True

No difference between kriged and not for the label